In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession

from shared.constants import DatasetPath

In [3]:
DATASET = DatasetPath('misc-football')

In [4]:
spark = (SparkSession.builder
         .appName(f'{DATASET}')
         .config('spark.sql.legacy.timeParserPolicy', 'LEGACY')
         .config("spark.executor.memory", "8g")
         .config("spark.driver.memory", "8g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "16g")
         .getOrCreate())

22/01/03 00:29:40 WARN Utils: Your hostname, megatron resolves to a loopback address: 127.0.1.1; using 192.168.1.89 instead (on interface enp7s0)
22/01/03 00:29:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/03 00:29:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/03 00:29:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/03 00:29:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/01/03 00:29:41 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/01/03 00:29:41 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [5]:
schema = T.StructType([
    T.StructField('src', T.IntegerType(), False),
    T.StructField('dst', T.IntegerType(), False),
    T.StructField('weight', T.IntegerType(), False),
])

df = (
    spark.read.csv(DATASET.raw_str('misc-football.mtx'), sep=' ', header=False, schema=schema, mode='DROPMALFORMED')
)
print(df.head(5))
print(df.count())

[Row(src=2, dst=1, weight=1), Row(src=5, dst=1, weight=1), Row(src=10, dst=1, weight=1), Row(src=17, dst=1, weight=1), Row(src=24, dst=1, weight=1)]
616


In [6]:
df_nodes = (
    df.select(F.col('src').alias('id'))
        .union(df.select(F.col('dst').alias('id')))
        .distinct()
)
df_nodes.head(5)

[Row(id=31), Row(id=85), Row(id=65), Row(id=53), Row(id=78)]

In [7]:
df_edges = (
    df.select(F.col('src'), F.col('dst'))
        .distinct()
)
df_edges.head(5)

[Row(src=56, dst=30),
 Row(src=113, dst=67),
 Row(src=105, dst=1),
 Row(src=59, dst=7),
 Row(src=62, dst=27)]

In [8]:
df_nodes.write.parquet(DATASET.processed_str('nodes_Team'), mode='overwrite')
df_edges.write.parquet(DATASET.processed_str('edges_PLAYS'), mode='overwrite')

In [9]:
from datasets.build_schema import build_schema

build_schema(
    spark,
    name=str(DATASET),
    nodes=[
        ('Team', DATASET.processed_str('nodes_Team')),
    ],
    edges=[
        ('Plays', 'Team', 'Team', DATASET.processed_str('edges_PLAYS')),
    ]
)

DatasetSchema(name='misc-football', prefix='MiscFootball', database='misc-football', description=None, nodes=[NodeSchema(label='Team', path='data/processed/misc-football/nodes_Team', properties=[Property(name='id', type='int', ignore=False, label=False)])], edges=[EdgeSchema(type='PLAYS', source='Team', target='Team', path='data/processed/misc-football/edges_PLAYS', properties=[Property(name='src', type='int', ignore=False, label=False), Property(name='dst', type='int', ignore=False, label=False)])])